----------------
# １．ライブラリのインポート

In [1]:
# 基本的なライブラリ
import numpy as np
import pandas as pd
from numpy.typing import NDArray
from scipy import stats

# Scikit-learn関連
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.utils.validation import check_X_y

# 抽象基底クラス (ABC)
from abc import ABCMeta, abstractmethod

# タイピングのサポート
from typing import Optional

# 可視化の設定
import matplotlib.pyplot as plt
import japanize_matplotlib
import itertools

plt.style.use("ggplot")

# 計算時間
import time

--------
# ２．実験設定

## 2.1 実験設定

In [2]:
### 実験データの設定 ###
n = 100000  # データサイズ
p = 80  # 特徴量数
s = 5  # 非ゼロ係数の数
rho = 0.35  # 相関レベル
snr = 0.5  # 信号対雑音比
beta_type = "type1"

### 実験設定 ###
SAMPLE_SIZE = 10000  # 標本サイズ
N_TRIALS = 10000  # 試行回数（標本平均を求める回数）
m_VALUE = 2  # 最適標本配分における各クラスタの最小標本数
RANDOM_STATE = 0  # 乱数シード
EXP_NUM = 50

### 実験方法 ###
# クラスタリング
CLUSTERING_METHOD = "kmeans"  # "gmm" or "kmeans" or "xmeans"
N_CLUSTERS = 6  # クラスタ数
K_MIN = 2
K_MAX = 10

# 特徴量選択
ALLOCATION_METHODS = [
    "Proportional",
    "Optimal",
]
SELECT_MAXIMUM_FEATURES = "yes"  # "yes" or "no"（特徴量数が MAXIMUM_FEATURES_TO_SELECT になるまで選ぶかいなか）
MAXIMUM_FEATURES_TO_SELECT = s  # 選択される最大の特徴量(特徴量選択ありの場合)


### 可視化 ###
TITLE_SIZE = 20
LABEL_SIZE = 15
TICK_SIZE = 12.5

### シード ###
np.random.seed(42)

In [3]:
if beta_type == "type1":
    BETA = [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]
    for i in range(p - 20):
        BETA.append(0)
    BETA = np.array(BETA)
    print(BETA)
ALL_FEATURES_INDEX = [i for i in range(p)]

[1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]


## 2.2 手法の名前

In [4]:
if CLUSTERING_METHOD == "kmeans":
    METHOD_NAME = "SFS-Kmeans-Var"

-----------
# ３．データの前処理

## 3.1 データ作成

In [5]:
def simulate_data(n, p, rho, snr, beta, seed):
    np.random.seed(seed)

    # 2. 共分散行列Σの生成
    Sigma = np.fromfunction(lambda i, j: rho ** np.abs(i - j), (p, p), dtype=int)

    # 3. Xの生成 (多変量正規分布)
    X = np.random.multivariate_normal(mean=np.zeros(p), cov=Sigma, size=n)

    # 4. 応答変数Yの生成
    signal_variance = beta.T @ Sigma @ beta  # β₀の分散
    noise_variance = signal_variance / snr  # ノイズ分散 σ²
    noise = np.random.normal(scale=np.sqrt(noise_variance), size=n)  # ノイズ項
    Y = X @ beta + noise

    return X, Y, beta

In [6]:
X_train, y_train, beta_train = simulate_data(
    n=n, p=p, rho=rho, snr=snr, beta=BETA, seed=0
)
X_test, y_test, beta_test = simulate_data(n=n, p=p, rho=rho, snr=snr, beta=BETA, seed=1)

## 3.5 標準化

In [7]:
sc = StandardScaler()
# 訓練データに基づいてfit
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

-------------
# ４．特徴量選択

In [8]:
TIME_DICT = {}
SELECTED_FEATURES_DICT = {}

### Allocation_in_Wrapper Class で特徴量選択

In [9]:
class Allocation_in_Wrapper(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        maximum_features_to_select: int,
        n_clusters: int,
        clustering_method: str = "kmeans",
        random_state: int = 0,
        select_maximum_features: str = "yes",
        k_min: int = 2,
        k_max: int = 10,
        allocation_method: str = "Proportional",
        sample_size: int = 10,
        n_trials: int = 100,
        m_value=2,
        M: Optional[NDArray] = None,
    ):
        self.maximum_features_to_select = maximum_features_to_select
        self.n_clusters = n_clusters
        self.clustering_method = clustering_method
        self.random_state = random_state
        self.select_maximum_features = select_maximum_features
        self.k_min = k_min
        self.k_max = k_max
        self.allocation_method = allocation_method
        self.sample_size = sample_size
        self.n_trials = n_trials
        self.m_value = m_value
        self.M = M

    def fss(self, X: pd.DataFrame, y: pd.DataFrame) -> "Allocation_in_Wrapper":
        X, y = check_X_y(X, y)
        n_all_features = X.shape[1]  # 総特徴量数

        features_score_dict = {}

        # 選ばれた特徴量と残っている特徴量の初期化
        current_features = []
        remaining_features = list(range(n_all_features))

        if self.select_maximum_features == "no":
            best_score = -np.inf

        while len(current_features) < self.maximum_features_to_select:
            best_feature = None  # 選ぶ特徴量の初期化

            if self.select_maximum_features == "yes":
                best_score = -np.inf

            for feature in remaining_features:
                temp_features = current_features + [
                    feature
                ]  # 特徴量をひとつ加え、score計算
                score, n_clusters = self.crit(X[:, temp_features], y)

                if score > best_score:
                    best_score = score
                    best_feature = feature
                    best_N_cluster_size = self.N_cluster_size
                    best_n_clusters = n_clusters
                    best_n_cluster_size = self.n_cluster_size

            if best_feature is not None:
                current_features.append(best_feature)
                remaining_features.remove(best_feature)
                num_of_features = len(current_features)
                print(
                    "num_of_features:",
                    num_of_features,
                    "current_features:",
                    current_features,
                    ", score:",
                    best_score,
                    "best_n_clusters:",
                    best_n_clusters,
                    "best_N_cluster_size:",
                    best_N_cluster_size,
                    "best_n_cluster_size:",
                    best_n_cluster_size,
                )

                features_score_dict[str(num_of_features)] = best_score  # 確認用
            else:
                break

        self.selected_features_index = current_features
        self.features_score_dict = features_score_dict

        return self

    def crit(self, X: pd.DataFrame, y: pd.DataFrame) -> float:
        # クラスタリング手法がGMMの場合
        if self.clustering_method == "gmm":
            model = GaussianMixture(
                n_components=self.n_clusters,
                random_state=self.random_state,
                init_params="kmeans",
            )
        # クラスタリング手法がKMEANSの場合
        if self.clustering_method == "kmeans":
            model = KMeans(
                n_clusters=self.n_clusters,
                random_state=self.random_state,
            )

        model.fit(X)
        self.N_cluster_label = model.predict(X)
        self.N_cluster_size = np.bincount(self.N_cluster_label)
        n_clusters = len(np.unique(self.N_cluster_label))

        if self.allocation_method == "Proportional":
            var = self.cauculate_var_proportional(X, y)
        if self.allocation_method == "Post":
            var = self.cauculate_var_proportional(X, y)
        if self.allocation_method == "Optimal":
            var = self.cauculate_var_optimal(X, y)
        score = -var

        return score, n_clusters

    def cauculate_var_proportional(self, X: pd.DataFrame, y: pd.DataFrame) -> float:
        self.n_cluster_size = self.proportional(X, y)
        H = np.where(self.n_cluster_size > 0)[0]
        N = np.sum(np.isin(self.N_cluster_label, H))
        S = np.array([np.var(y[self.N_cluster_label == h]) for h in H])
        n_h = self.n_cluster_size[H]
        N_h = self.N_cluster_size[H]
        var = (1 / int(N) ** 2) * (np.sum((N_h**2 * S) / n_h) - np.sum(N_h * S))
        return var

    def cauculate_var_optimal(self, X: pd.DataFrame, y: pd.DataFrame) -> float:
        self.n_cluster_size, H = self.optimal(X, y)
        N = np.sum(np.isin(self.N_cluster_label, H))
        S = np.array([np.var(y[self.N_cluster_label == h]) for h in H])
        n_h = self.n_cluster_size
        N_h = self.N_cluster_size[H]
        var = (1 / int(N) ** 2) * ((np.sum((N_h**2 * S) / n_h)) - np.sum(N_h * S))

        return var

    def proportional(self, X: pd.DataFrame, y: pd.DataFrame) -> NDArray:
        n_cluster_size: NDArray = np.round(
            self.N_cluster_size / self.N_cluster_size.sum() * self.sample_size
        ).astype(int)

        if n_cluster_size.sum() > self.sample_size:
            # nの合計がn_samplesより大きい場合は一番標本数が多いクラスタから削る
            n_cluster_size[np.argmax(n_cluster_size)] -= (
                n_cluster_size.sum() - self.sample_size
            )
        if n_cluster_size.sum() < self.sample_size:
            # nの合計がn_samplesより小さい場合は一番標本数が多いクラスタにたす
            n_cluster_size[np.argmax(n_cluster_size)] += (
                -n_cluster_size.sum() + self.sample_size
            )
        return n_cluster_size

    def optimal(self, X: pd.DataFrame, y: pd.DataFrame) -> NDArray:
        cluster_size_all = np.bincount(self.N_cluster_label)
        cluster_size_for_optimal = []
        labels_for_optimal = []
        for i in range(len(cluster_size_all)):
            if cluster_size_all[i] >= 2:
                cluster_size_for_optimal.append(cluster_size_all[i])
                labels_for_optimal.append(i)
        n_clusters_for_optimal = len(cluster_size_for_optimal)
        cluster_size_for_optimal = np.array(cluster_size_for_optimal)

        self.m = np.full(n_clusters_for_optimal, self.m_value)

        S = np.array([np.var(y[self.N_cluster_label == h]) for h in labels_for_optimal])
        d = (cluster_size_for_optimal**2) * S

        n_cluster_size = self.m.copy()  # 初期値
        M = self.M.copy() if self.M is not None else cluster_size_for_optimal.copy()
        I = np.arange(n_clusters_for_optimal)  # noqa #クラスタのインデックス

        while (n_cluster_size.sum() != self.sample_size) and len(I) != 0:
            delta = np.zeros(n_clusters_for_optimal)
            delta[I] = (d / (n_cluster_size + 1) - d / n_cluster_size)[I]
            h_star = np.argmin(delta[I])
            h_star = I[h_star]

            if n_cluster_size[h_star] + 1 <= M[h_star]:
                n_cluster_size[h_star] = n_cluster_size[h_star] + 1
            else:
                # Iの要素h_starを削除
                I_ = I.tolist()
                I_ = [i for i in I_ if i != h_star]
                I = np.array(I_)  # noqa

        # 制約チェック
        assert (
            n_cluster_size.sum() <= self.sample_size
        ), f"Total sample size is over than {self.sample_size}"
        assert np.all(
            n_cluster_size >= self.m
        ), "Minimum sample size constraint is not satisfied"
        if self.M is not None:
            assert np.all(
                n_cluster_size <= self.M
            ), "Maximum sample size constraint is not satisfied"

        return n_cluster_size, labels_for_optimal

    def get_selected_features_index(self):
        return self.selected_features_index  # 選択された特徴量のインデックス

    def get_features_score_dict(self):
        return self.features_score_dict

In [10]:
def process_allocation_in_wrapper(
    instance: "Allocation_in_Wrapper", X: NDArray, y: NDArray
) -> tuple[list, dict[int, float]]:
    instance.fss(X, y)
    selected_features_index = instance.get_selected_features_index()
    features_score_dict = instance.get_features_score_dict()
    selected_features_index = np.array(selected_features_index)

    return selected_features_index, features_score_dict

### インスタンスのリスト作成

In [11]:
instances = []
for allocation_method in ALLOCATION_METHODS:
    instances.append(
        (
            allocation_method,
            Allocation_in_Wrapper(
                maximum_features_to_select=MAXIMUM_FEATURES_TO_SELECT,
                n_clusters=N_CLUSTERS,
                clustering_method=CLUSTERING_METHOD,
                random_state=RANDOM_STATE,
                select_maximum_features=SELECT_MAXIMUM_FEATURES,
                k_min=K_MIN,
                k_max=K_MAX,
                allocation_method=allocation_method,
                sample_size=SAMPLE_SIZE,
                n_trials=N_TRIALS,
                m_value=m_VALUE,
            ),
        )
    )

### Allocaiton in Wrapper の実施

In [12]:
TIME_DICT = {}

# 各インスタンスに対して処理を実行
for allocation_method, instance in instances:
    time_list = []
    for exp_num in range(EXP_NUM):
        print("[", allocation_method, "]")
        start_time = time.time()
        _, _ = process_allocation_in_wrapper(instance, X_train_std, y_train)
        end_time = time.time()
        time_list.append(end_time - start_time)
    TIME_DICT[allocation_method] = time_list

[ Proportional ]
num_of_features: 1 current_features: [12] , score: -0.0012879462100293222 best_n_clusters: 6 best_N_cluster_size: [ 8470 23621 25391 16513 19552  6453] best_n_cluster_size: [ 847 2362 2540 1651 1955  645]
num_of_features: 2 current_features: [12, 8] , score: -0.001236738282510858 best_n_clusters: 6 best_N_cluster_size: [14941 25178 14710 14806 14991 15374] best_n_cluster_size: [1494 2518 1471 1481 1499 1537]
num_of_features: 3 current_features: [12, 8, 0] , score: -0.0012105318971677617 best_n_clusters: 6 best_N_cluster_size: [16717 16300 16962 16784 16651 16586] best_n_cluster_size: [1672 1630 1696 1678 1665 1659]
num_of_features: 4 current_features: [12, 8, 0, 16] , score: -0.0011817047163789843 best_n_clusters: 6 best_N_cluster_size: [14156 18334 17816 17783 17890 14021] best_n_cluster_size: [1416 1833 1782 1778 1789 1402]
num_of_features: 5 current_features: [12, 8, 0, 16, 4] , score: -0.0011970627586401794 best_n_clusters: 6 best_N_cluster_size: [16726 16589 16900

In [13]:
print(f"{p}:{TIME_DICT}")

80:{'Proportional': [56.9152455329895, 58.909085750579834, 58.15414214134216, 58.087013721466064, 57.92819333076477, 59.61953377723694, 60.79648518562317, 59.8999981880188, 58.83875632286072, 58.52111577987671, 58.18981862068176, 59.18365216255188, 61.02098798751831, 60.013916015625, 59.92047595977783, 60.642560958862305, 59.8067569732666, 60.98387145996094, 59.70317482948303, 60.67450451850891, 59.863513708114624, 60.53459453582764, 59.58225655555725, 60.03040170669556, 60.71448349952698, 60.18168234825134, 59.34932351112366, 58.522701263427734, 59.374324560165405, 58.11540246009827, 58.92206645011902, 58.35657000541687, 58.68981409072876, 59.028557538986206, 59.58184027671814, 59.324867486953735, 58.52444005012512, 60.147971391677856, 59.955528259277344, 59.164175033569336, 59.84628367424011, 58.19874978065491, 60.48573350906372, 59.04533886909485, 59.193652629852295, 59.623876333236694, 59.171881437301636, 58.39547634124756, 59.78320026397705, 59.11124038696289], 'Optimal': [205.910